# Relativistic Raytracer

or, a guide in three days of procrastination and a 30% grade deduction

*brought to you by @chas-card, @owl10124, and @TheRealOrange*

## Motivation

I don't think the motivation for a relativistic raytracer needs to be explained. It's a relativistic raytracer.